# Lab3 Classification and Regression

This week's exercise will be a bit shorter. The objective is to implement the core of the k-nn algorithm (most of the pre-processing and evaluation is given), followed by a Q&A moment

Schedule:
* Classify data using k-neighbors
* Use a confusion matrix to evluate models
* Q&A on the last lectures

## Reminders
* [GitHub repo](https://github.com/Faur/ITU-Data-Science-in-Games-Exercises)
* **Shut down notebooks** when you are done. Otherwise the server will run out of resources, and we will be forced to restart the them.
* Server storage is volatile! I.e. you must **save everything locally** that you don't want to loose.

In [ ]:
# ! git pull

In [ ]:
# Makes matplotlib plots work better with Jupyter
%matplotlib inline

# Import the necessary libraries. 
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing

## Take a look at the data

In [ ]:
# Check that data and data path is present
basedir = "../"
file = "fifa.csv"
assert os.path.isdir(f"{basedir}data") and os.path.exists(f"{basedir}data/{file}"), 'Data not found. Make sure to have the most recent version!'

data = pd.read_csv(f'{basedir}/data/fifa.csv', sep=",")

In [ ]:
sns.FacetGrid(data, height=5,hue="Position").map(plt.scatter,"SprintSpeed","Agility").add_legend()
sns.FacetGrid(data, height=5,hue="Position", col='Preferred Foot').map(plt.scatter,"ShotPower",'Penalties').add_legend()

The scatter plots shows that the `Position` of a player could be related to some of their statistics. The dataset contains 30+ statistics and we don't know which ones will be most helpful, so we are picking an arbitrary subset to avoid the [curse of dimensionality](https://en.wikipedia.org/wiki/Curse_of_dimensionality).
It's also possible to choose the best ones [algorithmically](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm#Feature_extraction)

In [ ]:
all_features = ['Crossing','Finishing','HeadingAccuracy','ShortPassing','Volleys','Dribbling','Curve','FKAccuracy','LongPassing',
            'BallControl','Acceleration','SprintSpeed','Agility','Reactions','Balance','ShotPower','Jumping','Stamina',
            'Strength','LongShots','Aggression','Interceptions','Positioning','Vision','Penalties','Composure',
            'Marking','StandingTackle','SlidingTackle','GKDiving','GKHandling','GKKicking','GKPositioning','GKReflexes']

features = ["SprintSpeed","Agility","ShotPower",'Penalties']
# cleaning: remove all the lines that contain a NaN in one of the feature columns
data = data.dropna(subset=features)

# Classification problem

0. pick a value for K (number of clusters) and N (number of neighbors)
1. split the data in train and validation set
2. normalize fields (in our case the data are already normalized)
3. foreach `datapoint` in `validation set`:
  1. find the N nearest neighbors
  2. set as label of `datapoint` the label that appears most between its neighbors

In [ ]:
# 0: define and N
K = 17

Choose the right K can be tricky. As usual, [stand on the shoulder of giants](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm#Parameter_selection)

# Step-by-step implemntation

In [ ]:
# 3.1: let's start with calculating the Euclidean distance from a single datapoint to all the other

knn_classification = lambda x, class_col, train_set: (
                                train_set[features].sub(x[features])
                               .pow(2).sum(1).pow(0.5)
                            )

# the first row of the result should be 0, since we are calculating the distance between a point and itself
# check pandas API for iloc[] if necessary
knn_classification(data.iloc[0], 'Position', data)

In [ ]:
# 3.2: to implement k-nn, we need the K closest neighbors. We'll get them with a pandas funciton,
# for convenience and performance

knn_classification = lambda x, class_col, train_set: (train_set[features].sub(x[features])
                               .pow(2).sum(1).pow(0.5)
                               .nsmallest(K)
                               )

knn_classification(data.iloc[0], 'Position', data)

In [ ]:
# 3.3: from here, we need the labels associated to this rows. The result is still referencing to our original dataset
# (the left column is the index on the original dataset), so we can use iloc[] as before for this

# let's also move everything we did up until now in a different function
nearest_neighbors = lambda x, train_set: (train_set[features].sub(x[features])
                               .pow(2).sum(1).pow(0.5)
                               .nsmallest(K)
                               ).index # IMPORTANT: .index was missing in the lab (see next cell)


knn_classification = lambda x, class_col, train_set: (
    # IMPORTANT: .loc[] instead of .iloc[] (see next cell)
    train_set.loc[nearest_neighbors(x, train_set)]
)


knn_classification(data.iloc[0], 'Position', data)

In [ ]:
# 3.4: now that we have the neigbors, lets get their labels and select the one that appear more times
nearest_neighbors = lambda x, train_set: (train_set[features].sub(x[features])
                               .pow(2).sum(1).pow(0.5)
                               .nsmallest(K)
                               ).index # IMPORTANT: .index was missing in the lab (see next cell)


knn_classification = lambda x, class_col, train_set: (
    # IMPORTANT: .loc[] instead of .iloc[] (see next cell)
    train_set.loc[nearest_neighbors(x, train_set)]
        [class_col]
        .mode()
)

# this time we are using a different datapoint, it will show us a problem
knn_classification(data.iloc[6], 'Position', data)

# Correction from the Lab. code

During the lab, `iloc[]` has been used insted of `loc[]`. They seems similar but actually they are QUITE different:
- `loc[i]` returns the row with index `i`
- `iloc[i]` returns the row in the `i`-th position

When creating a dataset (unless directly specified), the index of a row is just in position, so both functions will return the same row. That doesn't hold true after manipulating the DataFrame (eg, cleaning, sorting, etc).

The correct way to reference a dataset using the entries of a DataFrame `a` from the entries af another DataFrame `b` is:
```
a.loc[b.index]
```

In [ ]:
# 3.5: some of the modes that we will calculate will return more than one value. This won't work,
#      so we need to pick one of them. The best solution would be to assign a weight to each neighbor
#      based on its distance (see lecture's slides), sum the weights grouped by label and pick the label
#      with the highest score. You can try that or ask us for examples. Here we'll use the first entry,
#      even if this will degrade our performance (possubly by quite a lot)

nearest_neighbors = lambda x, train_set: (train_set[features].sub(x[features])
                               .pow(2).sum(1).pow(0.5)
                               .nsmallest(K)
                               ).index # IMPORTANT: .index was missing in the lab (see next cell)


knn_classification = lambda x, class_col, train_set: (
    # IMPORTANT: .loc[] instead of .iloc[] (see next cell)
    train_set.loc[nearest_neighbors(x, train_set)]
        [class_col]
        .mode()[0]
)


knn_classification(data.iloc[6], 'Position', data)

In [ ]:
def classify(dataset, class_col, random_state=None, frac=0.995):
    # 1: random split. It's always a good idea (maybe the dataset is sorted, and so on).
    # random_state that you to always get the same split (useful for testing). Check docs
    train_set = data.sample(frac=frac, random_state=random_state)
    valid_set = data.drop(train_set.index)

    classified_set = valid_set
    classified_set[f'Calculated {class_col}'] = valid_set.apply(
        lambda x: knn_classification(x, class_col, train_set),
        axis=1
    )
    return classified_set

classified_position = classify(data, 'Position')[['Position', 'Calculated Position']]
classified_position

# Binary classification with positive class
In many cases, it's important to determine if a datapoint belongs or not to a certain class or not and the distribution is skewed (poisonus/not poisonus, cancer/not cancer, ...)

In [ ]:
K = 3

data['Goalkeeper'] = data['Position'] == 'GK'

classified_goalkeeper = classify(data, 'Goalkeeper', frac=0.99)[['Goalkeeper', 'Calculated Goalkeeper']]

# Classification evaluation

1. generate confusion matrix
2. evaluate the classification

In [ ]:
from sklearn.metrics import confusion_matrix

def print_confusion_matrix(confusion_matrix, class_names, figsize = (10,7), fontsize=14):
    """Prints a confusion matrix, as returned by sklearn.metrics.confusion_matrix, as a heatmap.
    
    Arguments
    ---------
    confusion_matrix: numpy.ndarray
        The numpy.ndarray object returned from a call to sklearn.metrics.confusion_matrix. 
        Similarly constructed ndarrays can also be used.
    class_names: list
        An ordered list of class names, in the order they index the given confusion matrix.
    figsize: tuple
        A 2-long tuple, the first value determining the horizontal size of the ouputted figure,
        the second determining the vertical size. Defaults to (10,7).
    fontsize: int
        Font size for axes labels. Defaults to 14.
        
    Returns
    -------
    matplotlib.figure.Figure
        The resulting confusion matrix figure
    """
    df_cm = pd.DataFrame(
        confusion_matrix, index=class_names, columns=class_names, 
    )
    fig = plt.figure(figsize=figsize)
    try:
        heatmap = sns.heatmap(df_cm, annot=True, fmt="d")
    except ValueError:
        raise ValueError("Confusion matrix values must be integers.")
    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=fontsize)
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=fontsize)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    return fig


In [ ]:
def print_results(data, class_col, binary=False):
    labels = data[class_col].unique()
    cm = confusion_matrix(data[class_col], data[f'Calculated {class_col}'], labels=labels)

    print_confusion_matrix(
        cm,
        labels
    )
    plt.show()  # wait for the render (otherwise it will print the graphs async, after the strings)
    
    if binary:
        tn, fp, fn, tp = cm.ravel()

        accuracy = (tp + tn) / (tn + fp + fn + tp)
        err_rate = (fp + fn) / (tn + fp + fn + tp)
        sensitiv = (tp) / (fp + tp)
        specific = (tn) / (tn + fn)
        precision = (tp) / (tp + tn)

        print(cm.ravel())
        print(f'Accuracy: {accuracy}')
        print(f'Error rate: {err_rate}')
        print(f'Sensitivity: {sensitiv}')
        print(f'Specificity: {specific}')
        print(f'Precision: {specific}')
    else:
        diag = [cm[i][i] for i in range(len(cm))]        
        rows = [sum(cm[i]) for i in range(len(cm))]
        
        right_pred = sum(diag)
        wrong_pred = sum(rows)
        
        print(f'Precision: {right_pred / wrong_pred}')
    
        
    
    
print_results(classified_position, 'Position')
print_results(classified_goalkeeper, 'Goalkeeper', binary=True)

The accuracy seems high for the `Goalkeeper` class, but because the percentage of goalkeepers is low. In those cases, Snsitivity is a much better metric. Here will probably be very low because the metric are chosen arbitrarly. You can check the link above to see how to pick appropriate dimensions.

Additionally, for skewed distributions:
> A drawback of the basic "majority voting" classification occurs when the class distribution is skewed. That is, examples of a more frequent class tend to dominate the prediction of the new example, because they tend to be common among the k nearest neighbors due to their large number[[4]](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm#cite_note-Coomans_Massart1982-4). One way to overcome this problem is to weight the classification, taking into account the distance from the test point to each of its k nearest neighbors. The class (or value, in regression problems) of each of the k nearest points is multiplied by a weight proportional to the inverse of the distance from that point to the test point